In [ ]:
import numpy as np
from scipy.linalg import sqrtm
import math
from numba import jit
import matplotlib.pyplot as plt

In [ ]:
def quadratic(A, c, x):
    tau = 1.0  # Parameter tau
    lam = 1.0 # Parameter lambda
    d = x.shape[0]  # Dimension of x

    f0 = -tau * np.dot(c.T, x) + tau * lam * np.dot(x.T, np.dot(A.T, np.dot(A, x)))
    
    # ei ∈ Rd is the ith element of the canonical basis and 1 is row vector of all-ones
    # Canonical basis elements
    canonical_basis = np.eye(d)  
    # Row vector of all ones
    ones = np.ones(d)
    
    g = -np.sum(np.log(np.maximum(1e-15, np.dot(canonical_basis, x)))) - np.log(1 - np.dot(ones, x))

    return f0 + g


def gradient_f0(A, c, x, tau, lam):
    return -tau * c + 2 * tau * lam * A.T @ A @ x

def gradient_g(x):
    d = x.shape[0]
    canonical_basis = np.eye(d)  
    ones = np.ones(d)
    return - np.dot(canonical_basis.T, 1 / np.maximum(1e-15, np.dot(canonical_basis, x))) - ones / (1 - np.dot(ones, x))

def gradient_f(A, c, x):
    tau = 1.0  # Parameter tau
    lam = 1.0 # Parameter lambda
    grad_f0 = gradient_f0(A, c, x, tau, lam)
    grad_g = gradient_g(x)
    return grad_f0 + grad_g

def hessian_f0(A):
    tau = 1.0  # Parameter tau
    lam = 1.0 # Parameter lambda
    return tau * lam * np.dot(A.T, A)

def hessian_g(x):
    d = len(x)
    diag_inv = np.diag(1 / (x ** 2))
    ones = np.ones(d)
    return diag_inv + np.outer(ones, ones)

def hessian_f(A, x):
    tau = 1.0  # Parameter tau
    lam = 1.0 # Parameter lambda
    return hessian_f0(A) + hessian_g(x)

#### Original Newton's Method for Quadratic Programming

In [ ]:
# funtion for original newton's method to solve QP problem
def original_newton(A, c, x0, alpha, beta, max_iter=100000, eps=1e-2):
    '''
    This function implements the original Newton's update method on quadratic programming.
    A: constraint matrix for quadratic programming.
    c: linear term in the objective function.
    alpha: the step size reduction factor for line search, taking values between 0 and 1.
    beta: the sufficient decrease factor for line search.
    max_iter: maximum number of iterations.
    eps: tolerance for convergence.
    '''
    # Initialize variables
    n, d = A.shape
    xt = x0
    
    iteration = 0
    opt_gaps = []
    
    while iteration < max_iter:
        # Compute gradient and Hessian
        grad = gradient_f(A, c, xt)
        hessian = hessian_f(A, xt)

        # Compute the Newton sketch update direction
        direction = - np.dot(np.linalg.inv(hessian), grad)

        # Compute the Newton sketch decrement
        decrement = grad.T @ direction
        
        # Check convergence
        if (decrement**2)/2 <= eps:
            print(f"Converged in {iteration+1} iterations.")
            break
            

        # Compute optimality gap
        gap = np.linalg.norm(np.linalg.pinv(hessian) @ grad)
        opt_gaps.append(gap)

        # Line search to find the optimal step size
        # Initial step size
        mu = 0.05  
        aux_xt = xt + mu * direction
        
        while quadratic(A, c, aux_xt) > quadratic(A, c, xt) +  alpha * mu * gap:
            mu = beta*mu 
            aux_xt = xt + mu * direction         
        
        # Update solution
        xt += mu * direction
        iteration += 1
        if iteration >= max_iter:
                raise ValueError("Too many iterations")
    return opt_gaps

In [ ]:
# Define the quadratic programming problem
n = 5 # Number of constraints
d = 4 # Dimension of variables
A = np.random.randn(n, d)  # Constraint matrix
c = np.random.randn(d)  # Linear term
x0 = np.random.randn(d)  # Initial iterate
alpha = 0.5  # Step size reduction factor for line search
beta = 0.8  # Sufficient decrease factor for line search

# Solve the quadratic programming problem with Newton sketch method
opt_gaps_original = original_newton(A, c, x0, alpha, beta)

# Print the optimality gaps for each iteration
print("Optimality gaps for sketching Newton's Method with fixed sketch dimension:")
print(opt_gaps_original)

In [ ]:
# Plot opt_gaps by iteration
plt.plot(range(len(opt_gaps_original)), opt_gaps_original)
plt.xlabel('Iteration')
plt.ylabel('Optimality Gap')
plt.title("Optimality Gap by Iteration: Original Newton's Method")
plt.grid(True)
plt.show()

In [ ]:
# Plot opt_gaps by iteration

plt.figure(figsize=(8, 6))
plt.plot(range(len(opt_gaps_original)), opt_gaps_original, label = 'Original Newton(n=5)')
plt.plot(range(len(opt_gaps_newton5)), opt_gaps_newton5, label = 'Sketch Newton(n=5)')
#plt.plot(range(len(opt_gaps_newton_et)), opt_gaps_newton_et, label = 'Sketch Newton(e(t))')
plt.xlabel('Iteration')
plt.ylabel('Log Optimality Gap')
plt.title("Log Optimality Gap by Iteration: Original Newton vs Sketch Newton")
# Set the y-axis scale to logarithmic
plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

#### Sketching Newton's Method: fixed sketching accuracy

In [ ]:
@jit(nopython=True)
def fwht(a) -> None:
    h = 1
    while h < len(a):
        for i in range(0, len(a), h * 2):
            for j in range(i, i + h):
                a[j], a[j+h] = a[j]+a[j+h], a[j]-a[j+h]
        h *= 2
    return a

In [ ]:
# sketches based on randomized orthonormal systems (ROS)
def sketch_ros(X, m):
    
    # Set up 
    n, d = X.shape
    scalar = math.sqrt(n)
    index = np.random.randint(0, d, size = m) # index from 0 to n-1 (means 1-n)


    # Get PhiX, Phiy
    D = np.random.choice([-1, 1], size = n)
    
    PX = np.zeros((m, d))

    for i in range(d):
        D = np.random.choice([-1, 1], size = n)
        DX_i = D*X[:,i]
        HDX_i = fwht(DX_i)
        PX_i = scalar*np.random.choice(HDX_i, size = m)
        PX[:,i] = PX_i

    return(PX)

In [ ]:
def newton_sketch(A, c, m, x0, alpha, beta, max_iter=1000000, eps=1e-2):
    '''
    This function implements the Newton sketch method on quadratic programming.
    A: constraint matrix for quadratic programming.
    c: linear term in the objective function.
    m: sketch dimension.
    alpha: the step size reduction factor for line search, taking values between 0 and 1.
    beta: the sufficient decrease factor for line search.
    max_iter: maximum number of iterations.
    eps: tolerance for convergence.
    '''
    # Initialize variables
    n, d = A.shape
    xt = x0
    
    iteration = 0
    opt_gaps = []
    
    while iteration < max_iter:
        # Compute gradient and Hessian
        # Compute gradient and Hessian
        grad_f = gradient_f(A, c, xt)
        h_f = hessian_f(A, xt)
        
        # Sketch the gradient and Hessian
        sketch_hessian = sketch_ros(sqrtm(h_f), m)  # Generate sketched hessian matrix

        
        # Compute the approximate Newton step
        delta_xt = - np.linalg.pinv(sketch_hessian.T @ sketch_hessian) @ grad_f
        # Compute the approximate Newton decrement
        lambda_xt = grad_f.T @ delta_xt
        
        # Check convergence
        if (lambda_xt**2)/2 <= eps:
            print(f"Converged in {iteration+1} iterations.")
            break

        # Compute optimality gap
        gap = np.linalg.norm(np.linalg.pinv(h_f) @ grad_f)
        opt_gaps.append(gap)

        # Line search to find the optimal step size
        # Initial step size
        mu = 0.05
        aux_xt = xt + mu * delta_xt
        
        while quadratic(A, c, aux_xt) > quadratic(A, c, xt) + alpha * mu * gap:
            mu = beta * mu 
            aux_xt = xt + mu * delta_xt
        
        # Update solution
        xt += mu * delta_xt
        iteration += 1
        
        if iteration >= max_iter:
            raise ValueError("Too many iterations")
    
    return opt_gaps


In [ ]:
# Define the quadratic programming problem
n = 5  # Number of constraints
d = 4 # Dimension of variables
A = np.random.randn(n, d)  # Constraint matrix
c = np.random.randn(d)  # Linear term
m = 4*d  # Sketch dimension
x0 = np.random.randn(d)  # Initial iterate
alpha = 0.5  # Step size reduction factor for line search
beta = 0.8  # Sufficient decrease factor for line search

# Solve the quadratic programming problem with Newton sketch method
opt_gaps_newton5 = newton_sketch(A, c, m, x0, alpha, beta)

# Print the optimality gaps for each iteration
print("Optimality gaps for sketching Newton's Method with fixed sketch dimension:")
print(opt_gaps_newton5)

In [ ]:
# Define the quadratic programming problem
n = 10  # Number of constraints
d = 4 # Dimension of variables
A = np.random.randn(n, d)  # Constraint matrix
c = np.random.randn(d)  # Linear term
m = 4*d  # Sketch dimension
x0 = np.random.randn(d)  # Initial iterate
alpha = 0.5  # Step size reduction factor for line search
beta = 0.8  # Sufficient decrease factor for line search

# Solve the quadratic programming problem with Newton sketch method
opt_gaps_newton10 = newton_sketch(A, c, m, x0, alpha, beta)

# Print the optimality gaps for each iteration
print("Optimality gaps for sketching Newton's Method with fixed sketch dimension:")
print(opt_gaps_newton10)

In [ ]:
# Define the quadratic programming problem
n = 15  # Number of constraints
d = 4 # Dimension of variables
A = np.random.randn(n, d)  # Constraint matrix
c = np.random.randn(d)  # Linear term
m = 6*d  # Sketch dimension
x0 = np.random.randn(d)  # Initial iterate
alpha = 0.5  # Step size reduction factor for line search
beta = 0.8  # Sufficient decrease factor for line search

# Solve the quadratic programming problem with Newton sketch method
opt_gaps_newton15 = newton_sketch(A, c, m, x0, alpha, beta)

# Print the optimality gaps for each iteration
print("Optimality gaps for sketching Newton's Method with fixed sketch dimension:")
print(opt_gaps_newton15)

In [ ]:
# Define the quadratic programming problem
n = 20  # Number of constraints
d = 4 # Dimension of variables
A = np.random.randn(n, d)  # Constraint matrix
c = np.random.randn(d)  # Linear term
m = 6*d  # Sketch dimension
x0 = np.random.randn(d)  # Initial iterate
alpha = 0.5  # Step size reduction factor for line search
beta = 0.8  # Sufficient decrease factor for line search

# Solve the quadratic programming problem with Newton sketch method
opt_gaps_newton20 = newton_sketch(A, c, m, x0, alpha, beta)

# Print the optimality gaps for each iteration
print("Optimality gaps for sketching Newton's Method with fixed sketch dimension:")
print(opt_gaps_newton20)

In [ ]:
# Plot opt_gaps by iteration
plt.figure(figsize=(8, 6))

plt.plot(range(len(opt_gaps_newton5)), opt_gaps_newton5, label = 'Sketch Newton(n=5)')
plt.plot(range(len(opt_gaps_newton10)), opt_gaps_newton10, label = 'Sketch Newton(n=10)')
plt.plot(range(len(opt_gaps_newton15)), opt_gaps_newton15, label = 'Sketch Newton(n=15)')
plt.plot(range(len(opt_gaps_newton20)), opt_gaps_newton20, label = 'Sketch Newton(n=20)')
plt.xlabel('Iteration')
plt.ylabel('Log Optimality Gap')
plt.title('Log Optimality Gaps by Iteration for Newton Sketch')
plt.grid(True)
plt.yscale('log')
plt.legend()
plt.show()